# QfDate Example

## Basics

QfDate is the QuantForm library's date object. It is mainly built on top of the QuantLib Date object with many quality of life improvements.

In [1]:
# Setup for the notebook
from quantform.pylib import QfDate
QfDate

quantform.pylib.QfDate.QfDate

The QfDate object functions very similarly to Python's datetime objects. It can be initialized simply with the year, month and day of interest (in that order)

In [2]:
date = QfDate(2025, 8, 1)
print(date)

2025-08-01


However, the QfDate also takes optional keyword arguments the name of the used calendar and the day count convention. The default ones are 'Frankfurt' and 'Business/252' respectively

In [3]:
print(repr(date))

Date: 2025-08-01
Convention: Business/252
Calendar: Frankfurt


These are generally good options for most cases. If the focus on trading is in US market then the calendar 'NYSE' should be preferred and similarly if the focus is on the UK market then the calendar 'London' should be utilized. Note that not all of the calendars available in QuantLib are currently supported.

If the trading focuses on fixed-income securities, the day count convention should be changed to either 'ACT/360' or 'ACT/365'.

The values used in initialization can also be accessed as properties of the object

In [4]:
print(f"Year: {date.year}")
print(f"Month: {date.month}")
print(f"Day: {date.day}")
print(f"Calendar: {date.calendar}")
print(f"Convention: {date.convention}")

Year: 2025
Month: 8
Day: 1
Calendar: Frankfurt
Convention: Business/252


The important feature of the calendar is that it allows distinguishing if a given date is a bank day or not. This can be done with the 'is_prod_date' method

In [5]:
print(f"{date} is a production date: {date.is_prod_date()}")
# The date '2025-08-01 is a Friday
print(f"{date.date_shift(1)} is a production date: {(date + 1).is_prod_date()}")

2025-08-01 is a production date: True
2025-08-02 is a production date: False


Likewise the next and previous bank days can be found with methods 'next_prod_date' and 'prev_prod_date' respectively

In [6]:
print(f"Next production date from {date}: {date.next_prod_date()}")
print(f"Previous production date from {date}: {date.prev_prod_date()}")

Next production date from 2025-08-01: 2025-08-04
Previous production date from 2025-08-01: 2025-07-31


Simple iteration over the production dates makes it possible to calculate how many of them are between two dates. This is implemented as 'prod_days_since' and 'prod_days_until' methods.

In [7]:
print(f"Number of production dates between {date} and {date.date_shift(100)}: {date.prod_days_until(date.date_shift(100))}")

Number of production dates between 2025-08-01 and 2025-11-09: 71


Other useful methods can be the various '*_year_start', '*_year_end', '*_month_start', '*_month_end' methods. As is implicit in the names, these methods return the start or end of the specified month or year. As an example

In [8]:
print(f"The first day of the year for {date}: {date.this_year_start()}")
print(f"The last day of the next month for {date}: {date.next_month_end()}")

The first day of the year for 2025-08-01: 2025-01-01
The last day of the next month for 2025-08-01: 2025-09-30


## Date manipulation

The addition and subtraction as well as multiplication and division operators have been overloaded to provide useful date manipulation methods. 

Adding and subtracting integers changes the day under the given day count convention. That is adding 252 days under 'Business/252' convention should be equivalent to moving forward roughly one year, but under 'ACT/365' convention would be equal to moving forward exactly 252 days. Equivalent holds for subtraction.

In [9]:
bus_252_date = date
act_365_date = QfDate(2025, 8, 1, convention="ACT/365")

print(repr(bus_252_date + 252), end="\n\n")
print(repr(act_365_date + 252))

Date: 2026-08-01
Convention: Business/252
Calendar: Frankfurt

Date: 2026-04-10
Convention: ACT/365
Calendar: Frankfurt


The multiplication and division operators provide a way to manipulate the date with a year fraction. Assume the given date is multiplied by some number $x$. The returned date would then be 

$$
  d_2 = d_1 + x \langle\text{days in a year}\rangle
$$

where $\langle\text{days in a year}\rangle$ is determined by the day count convention. Division works equivalently, but subtracts the number of days.

In [10]:
print(repr(bus_252_date * 1.), end="\n\n")
print(repr(act_365_date * 0.5), end="\n\n")

Date: 2026-08-01
Convention: Business/252
Calendar: Frankfurt

Date: 2026-01-31
Convention: ACT/365
Calendar: Frankfurt



In addition to the addition and subtraction operators, which do the calculations under the day count convention, a simple day shift is also included as method 'date_shift'. As the name suggests this method shifts the date by the specified amount without any complications

In [11]:
print(repr(bus_252_date.date_shift(3)))

Date: 2025-08-04
Convention: Business/252
Calendar: Frankfurt


Dates can be compared to one another if wanted to determine which is further in the future

In [12]:
print(bus_252_date + 3 > bus_252_date)

True


However, only comparison between dates with the same calendar and convention is allowed

In [13]:
try:
  act_365_date > bus_252_date
except AssertionError as e:
  print(e)

The conventions must match! (ACT/365 != Business/252)


This behaviour holds for all methods where the second argument is also a QfDate instance

A very common number needed is the year fraction between two dates. This can be calculated with the 'timedelta' method

In [14]:
bus_252_date2 = bus_252_date + 100
print(f"Year fraction: {bus_252_date.timedelta(bus_252_date2)}")
print(f"Should be roughly: {100 / 252}")

Year fraction: 0.4087301587301587
Should be roughly: 0.3968253968253968
